In [4]:
import tensorflow as tf

In [5]:
train_ds = tf.data.Dataset.load('asl_train_data')
train_batches = train_ds.batch(64).prefetch(buffer_size=tf.data.AUTOTUNE)
train_batches

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 120), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

Three layers BiLSTM model

In [6]:
class aslModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.pre_dense0 = tf.keras.layers.Dense(512)
    self.lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences = True))
    self.lstm2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences = True))
    self.lstm3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256))
    self.dense1 = tf.keras.layers.Dense(512)
    self.dense2 = tf.keras.layers.Dense(512, activation='relu')
    self.final = tf.keras.layers.Dense(250)

  def call(self, inputs):
    x = self.pre_dense0(inputs)
    x = self.lstm1(x)
    x = self.lstm2(x)
    x = self.lstm3(x)
    x = self.dense1(x)
    x = self.dense2(x)
    x = self.final(x)
    return x

In [7]:
sample_model = aslModel()
for (input, target) in train_batches.take(1):
    print(target.shape)
    out = sample_model(input)
    print(out.shape)

(64,)
(64, 250)


In [20]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=8000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

learning_rate = CustomSchedule(1024)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)


In [21]:
model = aslModel()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=optimizer,
              metrics=['accuracy'])

In [22]:
model.fit(train_batches, epochs=10)

Epoch 1/10


2023-04-02 20:29:16.762826: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-02 20:29:17.361618: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-02 20:29:17.379896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-02 20:29:17.630822: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-02 20:29:17.648536: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-02 20:29:17.877781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-04-02 20:29:17.894337: I tensorflow/core/grappler/optimizers/cust

1344/1344 [==============================] - 219s 158ms/step - loss: 5.5190 - accuracy: 0.0042
Epoch 2/10
1344/1344 [==============================] - 210s 156ms/step - loss: 5.3469 - accuracy: 0.0112
Epoch 3/10
1344/1344 [==============================] - 213s 158ms/step - loss: 5.0549 - accuracy: 0.0289
Epoch 4/10
1344/1344 [==============================] - 208s 155ms/step - loss: 4.8163 - accuracy: 0.0534
Epoch 5/10
1344/1344 [==============================] - 209s 155ms/step - loss: 4.5960 - accuracy: 0.0839
Epoch 6/10
1344/1344 [==============================] - 209s 156ms/step - loss: 4.4160 - accuracy: 0.1175
Epoch 7/10
1344/1344 [==============================] - 207s 154ms/step - loss: 4.2677 - accuracy: 0.1454
Epoch 8/10
1344/1344 [==============================] - 207s 154ms/step - loss: 4.1135 - accuracy: 0.1788
Epoch 9/10
1344/1344 [==============================] - 208s 154ms/step - loss: 3.9840 - accuracy: 0.2052
Epoch 10/10
1344/1344 [==============================] - 

In [23]:
model.fit(train_batches, epochs=10)

Epoch 1/10
1344/1344 [==============================] - 211s 157ms/step - loss: 3.7873 - accuracy: 0.2490
Epoch 2/10
1344/1344 [==============================] - 208s 155ms/step - loss: 3.7158 - accuracy: 0.2649
Epoch 3/10
1344/1344 [==============================] - 207s 154ms/step - loss: 3.6453 - accuracy: 0.2812
Epoch 4/10
1344/1344 [==============================] - 207s 154ms/step - loss: 3.5957 - accuracy: 0.2933
Epoch 5/10
1344/1344 [==============================] - 207s 154ms/step - loss: 3.5271 - accuracy: 0.3098
Epoch 6/10
1344/1344 [==============================] - 210s 157ms/step - loss: 3.4887 - accuracy: 0.3183
Epoch 7/10
1344/1344 [==============================] - 211s 157ms/step - loss: 3.4332 - accuracy: 0.3324
Epoch 8/10
1344/1344 [==============================] - 211s 157ms/step - loss: 3.3944 - accuracy: 0.3424
Epoch 9/10
1344/1344 [==============================] - 210s 156ms/step - loss: 3.3527 - accuracy: 0.3533
Epoch 10/10
1344/1344 [=======================